In [1]:
import os
import json
import pandas as pd
import getpass
import re

from langchain_openai import ChatOpenAI
from uuid import uuid4
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, ToolMessage, AIMessage, RemoveMessage
from langchain.output_parsers import PydanticOutputParser
from langchain.prompts import PromptTemplate
from langgraph.graph import StateGraph, START, END
from langchain.tools import BaseTool, StructuredTool, tool

from langgraph.checkpoint.sqlite import SqliteSaver

from pydantic import BaseModel, Field
from typing import Literal, Dict, Tuple, List, Optional, TypedDict, Annotated

from langchain_community.tools.tavily_search import TavilySearchResults
import operator

# from llm_utils import get_llm

import itertools
import ast
from IPython.display import display, Markdown
import gradio as gr

import time

from dotenv import load_dotenv
load_dotenv("/home/jovyan/Albert/.env")

os.environ["OPENAI_API_KEY"] = os.environ.get("OPENAI_API_KEY")
# os.environ["SLIP_AUTH_URL"] = os.environ.get("SLIP_AUTH_URL")
# os.environ["SLIP_USERNAME"] =os.environ.get("SLIP_USERNAME")
# os.environ["SLIP_PASSWORD"] =  os.environ.get("SLIP_PASSWORD")
# os.environ["SLIP_CLIENT_ID"]= os.environ.get("SLIP_CLIENT_ID")
# os.environ["SLIP_APP_NAME"]=os.environ.get("SLIP_APP_NAME")

/home/jovyan/envs/llm_agents/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# We will save memory with sqlite saver
os.environ["TAVILY_API_KEY"] = "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"
tavily = TavilySearchResults(max_results=2)

## Format Data from CSV

In [3]:
df=pd.read_csv('/home/jovyan/data/LLM/EMR_LLM_data_combo_2204mcids_O.csv')

/home/jovyan/tmp/ipykernel_271395/295251058.py:1: DtypeWarning: Columns (4,6,7) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv('/home/jovyan/data/LLM/EMR_LLM_data_combo_2204mcids_O.csv')


In [4]:
df['type'] = df['type'].replace('gpi', 'rx')

In [5]:
df.head()

,mcid,dos,codeIndex,type,value,code,codeDesc,abnormal_cd,id
0,667208490,2019-08-02,84847.0,rx,NaN,42200032,Fluticasone,NaN,27489.0
1,667208490,2019-08-12,85106.0,rx,NaN,58160040,Fluoxetine,NaN,27489.0
2,667208490,2019-08-12,85356.0,rx,NaN,73200010,Amantadine,NaN,27489.0
3,667208490,2019-08-19,84352.0,rx,NaN,16990002,Anti-infective Misc. Combination - Two Ingredient,NaN,27489.0
4,667208490,2019-08-19,85343.0,rx,NaN,72600030,Gabapentin,NaN,27489.0


In [6]:
mcidList = list(df["mcid"].unique())
print(len(mcidList))

2204


In [7]:
# mcids = [609394282, 409434803, 608416755, 640516650]
code_rows = df[df["mcid"].isin(mcidList) & df["codeIndex"].notna()]

# Selecting rows where "mcid" is in the given list and "codeIndex" is NaN
emr_rows = df[df["mcid"].isin(mcidList) & df["codeIndex"].isna() & df["dos"].notna()]

gender_rows = df[df["mcid"].isin(mcidList) & df["codeIndex"].isna() & df["dos"].isna()]

In [8]:
# The normal mapping between ICD10 and ICD10 full description
lookup_df_diag = pd.read_csv('/home/jovyan/data/ihan_paper/data_v2/codeDict/codeDict_diag_claimEMR_hcc138.csv')
dict_obj_diag = lookup_df_diag[['cd_value', 'ICD_FULL_DESC']].dropna().set_index('cd_value')['ICD_FULL_DESC'].to_dict()

## Testing Agents
This will be multiple code agents that looks at the diag, proc, rx, and lab lookup table and history agnostically to get a summary, self-assigned LLM Q and A, and basic Q and A for the patient.

### Tools

In [9]:
from typing import List, Dict

@tool
def returnSpecificICD10DatesCodes(patient_codes_list: List[List[str]], dates_list: List[str], codes_disease_dictionary_mapping: Dict[str, str], diseaseSearch: str) -> Tuple[List[str], List[List[str]], List[str]]:
    """returns the patient's specific ICD10 codes and dates for the disease searched based on their existing dates and codes."""
    # Extract the specific disease codes from the dictionary mapping
    check = set([code for code, disease in codes_disease_dictionary_mapping.items() if disease == diseaseSearch])

    filtered_dates = []
    filtered_code_lists = []
    specificDiseaseCodesFound = set()

    # Filtering the dates and code lists based on the disease search
    for i, codes in enumerate(patient_codes_list):
        intersection = set(codes) & check
        if intersection:
            filtered_code_lists.append(list(intersection))
            filtered_dates.append(dates_list[i])
            specificDiseaseCodesFound.update(intersection)

    return filtered_dates, filtered_code_lists, list(specificDiseaseCodesFound)

In [10]:
# create codes in alphanumeric order.

# lookup_dict is like : Dict[str, str]
@tool
def createLookupTable(codes: List[List[str]], lookup_dict) -> str:
    """Create the codes and descriptions lookup table for the specific patient."""
    
    
    print(codes)
    print(lookup_dict)
    
    flattened_list = [item for sublist in codes for item in sublist]
    
    # get unique codes, which are keys in lookup_dict
    unique_codes = set(flattened_list) & lookup_dict.keys()
    # print("Unique Codes 777", unique_codes)
    
    # sort codes in alphanumeric order
    sorted_unique_codes = sorted(unique_codes)
    
    # join each code and its description in a string, separated by ":"
    formatted_string = " \n ".join(f"{code}: {lookup_dict[code]}" for code in sorted_unique_codes)
    
    # print("FORMATTED STRING", formatted_string)

    return formatted_string

In [11]:
from typing import List, Dict

@tool
def createLabLookupTable(codes: List[Dict], lookup_dict: Dict) -> str:
    """Create the codes and descriptions lookup table for the specific patient."""
    
    
    # Flatten dict to create list of all codes
    flattened_list = [key for sublist in codes for key in sublist]
    
    # get unique codes, which are keys in lookup_dict
    unique_codes = set(flattened_list) & set(lookup_dict.keys())
    
    print("Unique Codes 777", unique_codes)
    
    # sort codes in alphanumeric order
    sorted_unique_codes = sorted(unique_codes)
    
    # join each code and its description in a string, separated by ":"
    formatted_string = " \n ".join(f"{code}: {lookup_dict[code]}" for code in sorted_unique_codes)
    
    print(formatted_string)
    
    return formatted_string


In [12]:
@tool
def createLabCodeHistory(dates: List[str], codes: List[Dict]) -> str:
    """Create the visit dates and medical codes history list for the specific patient."""
    return " \n ".join(
    f"{date}: [{', '.join(f'{k}: {v}' for k, v in code.items())}]"
    for date, code in zip(dates, codes)
    )

In [13]:
@tool
def createCodeHistory(dates: List[str], codes: List[List[str]]) -> str:
    """Create the visit dates and medical codes history list for the specific patient."""
    return ' \n '.join(
       f"{date}: {', '.join(map(str, codes))}" 
       for date, codes in zip(dates, codes)
    )


In [14]:
import datetime as dt
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import base64
import io
from PIL import Image as PIL_Image

def try_float(val, default=False):
    """
    Try to convert a value to float.
    Return the float value on success, 'default' value on fail.
    """
    try:
        return float(val)
    except ValueError:
        return default

def labVizData(lab_code: str, lab_description: str, lab_unit_measurement: str, plot_name: str, dates, lab_values):
    # Initialize lists to store the code values, associated dates, and codes.
    code_values = []
    date_values = []

    # Iterate over each dictionary in the lab_values list along with the index
    for i, lab_dict in enumerate(lab_values):
        # For each dictionary, fetch the first key and value (assuming each dictionary has only one pair).
        # Take care of the case if the dictionary is empty.
        if lab_code in lab_dict:
            value = try_float(lab_dict[lab_code])  # Assuming try_float is a function that attempts to convert a string to a float and returns None or False if it fails.        
            if value is not False:   
                # Append to code_values, dates, and codes lists
                code_values.append(value)
                date_values.append(dates[i])

    if len(code_values) > 0:
        ax = plt.gca()
        x_axis = [dt.datetime.strptime(d,'%Y-%m-%d').date() for d in date_values]
        ax.xaxis.set_major_formatter(mdates.DateFormatter('%m-%d-%y'))
        ax.set_xticks(x_axis)
        plt.xlabel("Dates")
        plt.ylabel(lab_unit_measurement)
        plt.title(lab_code+" (" + lab_description + ")")
        plt.plot(x_axis, code_values, '--bo')
        
        plt.savefig(plot_name, format="png")
        plt.close()
    else:
        return "Patient does not have any "+ lab_code + " data."
    
    return "![" + lab_description + "](" + plot_name+")"

@tool
def labViz(lab_code: str, lab_description: str, lab_unit_measurement: str, plot_name: str):
    """A lab test graphing visualization tool patient's medical history. lab_code must be the exact lab_code name. \ 
Will save an image in the format where plot_name looks like 'plot_name.png'. \
Useful for when you need to answer questions with a visualization of a specific lab test's values. \ 
Inputs are lab_code (the lab test code name), lab_description (description/name of lab test), \
lab_unit_measurement (unit of measurement for lab values), plot_name (save the plot as plot_name.png)"""
        
    return lab_code, lab_description, lab_unit_measurement, plot_name


## Prompts

In [15]:
DIAG_SUMMARY_PROMPT = """
Instructions:
1. Output the JSON string based on the "Patient Diag Code Table" using the provided "Output Format Json."
2. Organize the most relevant codes together in specific categories that provide value to the health professional reviewing them.
3. Ensure the JSON string is correctly formatted without extra characters or markdown.
4. Include all codes from the "Patient Diag Code Table."
5. Prioritize more serious categories and conditions at the beginning of the JSON output

Input Format for "Patient Diag Code Table":
'''
diagCode1: description of diag code 1
diagCode2: description of diag code 2
etc.
'''

Patient Diag Code Table:
{lookupTable}

Output Format Json:

{{
  "diagCategories": [
    {{
      "category": "Category Name 1",
      "values": [
        {{
          "name": "Diag Name 1",
          "code": "diagCode1"
        }},
        {{
          "name": "Diag Name 2",
          "code": "diagCode2"
        }}
      ]
    }},
    {{
      "category": "Category Name 2",
      "values": [
        {{
          "name": "Diag Name 3",
          "code": "diagCode3"
        }},
        {{
          "name": "Diag Name 4",
          "code": "diagCode4"
        }}
      ]
    }}
    # Continue with additional categories and values as necessary
  ]
}}
""".strip()


In [16]:
RX_SUMMARY_PROMPT = """
Instructions:
1. Output the JSON string based on the "Patient Rx Code Table" using the provided "Output Format Json."
2. Organize the most relevant codes together in categories.
3. Ensure the JSON string is correctly formatted without extra characters or markdown.
4. Include all codes from the "Patient Rx Code Table."
5. Prioritize more serious categories and conditions at the beginning of the JSON output

Input Format for "Patient Rx Code Table":
'''
rxCode1: description of rx code 1
rxCode2: description of rx code 2
etc.
'''

Patient Rx Code Table:
{lookupTable}

Output Format Json:

{{
  "rxCategories": [
    {{
      "category": "Category Name 1",
      "values": [
        {{
          "name": "Rx Name 1",
          "code": "rxCode1"
        }},
        {{
          "name": "Rx Name 2",
          "code": "rxCode2"
        }}
      ]
    }},
    {{
      "category": "Category Name 2",
      "values": [
        {{
          "name": "Rx Name 3",
          "code": "rxCode3"
        }},
        {{
          "name": "Rx Name 4",
          "code": "rxCode4"
        }}
      ]
    }}
    # Continue with additional categories and values as necessary
  ]
}}
""".strip()


In [17]:
PROC_SUMMARY_PROMPT = """
Instructions:
1. Output the JSON string based on the "Patient Proc Code Table" using the provided "Output Format Json."
2. Organize the most relevant codes together in categories.
3. Ensure the JSON string is correctly formatted without extra characters or markdown.
4. Include all codes from the "Patient Proc Code Table."
5. Prioritize more serious categories and conditions at the beginning of the JSON output

Input Format for "Patient Proc Code Table":
'''
procCode1: description of proc code 1
procCode2: description of proc code 2
etc.
'''

Patient Proc Code Table:
{lookupTable}

Output Format Json:

{{
  "procCategories": [
    {{
      "category": "Category Name 1",
      "values": [
        {{
          "name": "proc Name 1",
          "code": "procCode1"
        }},
        {{
          "name": "proc Name 2",
          "code": "procCode2"
        }}
      ]
    }},
    {{
      "category": "Category Name 2",
      "values": [
        {{
          "name": "proc Name 3",
          "code": "procCode3"
        }},
        {{
          "name": "proc Name 4",
          "code": "procCode4"
        }}
      ]
    }}
    # Continue with additional categories and values as necessary
  ]
}}
""".strip()


In [18]:
LAB_SUMMARY_PROMPT = """
Instructions:
1. Output the JSON string based on the "Patient Lab Code Table" using the provided "Output Format Json."
2. Organize the most relevant codes together in categories.
3. Ensure the JSON string is correctly formatted without extra characters or markdown.
4. Include all codes from the "Patient Lab Code Table."
5. Prioritize more serious categories and conditions at the beginning of the JSON output

Input Format for "Patient Lab Code Table":
'''
labCode1: description of lab code 1
labCode2: description of lab code 2
etc.
'''

Patient Lab Code Table:
{lookupTable}

Output Format Json:

{{
  "labCategories": [
    {{
      "category": "Category Name 1",
      "values": [
        {{
          "name": "lab Name 1",
          "code": "labCode1",
        }},
        {{
          "name": "lab Name 2",
          "code": "labCode2"
        }}
      ]
    }},
    {{
      "category": "Category Name 2",
      "values": [
        {{
          "name": "lab Name 3",
          "code": "labCode3"
        }},
        {{
          "name": "lab Name 4",
          "code": "labCode4"
        }}
      ]
    }}
    # Continue with additional categories and values as necessary
  ]
}}
""".strip()


## Agent States

In [19]:
def reduce_messages(left: list[AnyMessage], right: list[AnyMessage]) -> list[AnyMessage]:
    
    print("left", left)
    print("right", right)
    print()
    print("-----------------")
    # assign ids to messages that don't have them
    for message in right:
        if not message.id:
            message.id = str(uuid4())
    # merge the new messages with the existing messages
    merged = left.copy()
    
    print("MERGED")
    for message in right:
        print(message)
        for i, existing in enumerate(merged):
            # replace any existing messages with the same id
            if existing.id == message.id:
                merged[i] = message
                break
        else:
            # append any new messages to the end
            merged.append(message)
            
    print("END OF MERGED")
    return merged

class AgentState(TypedDict):
    mcid: int
    verbose: int
    
    lookup_dict_diag: Dict[str, str]
    dates_diag: List[List[str]]
    codes_diag: List[List[int]]
    
    lookup_dict_rx: Dict[str, str]
    dates_rx: List[List[str]]
    codes_rx: List[List[int]]
    
    lookup_dict_proc: Dict[str, str]
    dates_proc: List[List[str]]
    codes_proc: List[List[int]]
    
    lookup_dict_lab: Dict[str, str]
    dates_lab: List[List[str]]
    codes_lab: List[Dict[str, str]]
    
    EMR_Notes: str
    
    messages: Annotated[List[AnyMessage], reduce_messages]
    

## Summary Agent Graph

In [20]:
class SummaryAgent():
    def __init__(self, model, tools, checkpointer, system=""):
        self.system = system
        
        graph = StateGraph(AgentState)
        # Diag Code Related
        graph.add_node("Diagnosis Summary", self.diag_general_summary)
        
        # Rx Code Related
        graph.add_node("Rx Summary", self.rx_general_summary)

        # Proc Code Related
        graph.add_node("Procedure Summary", self.proc_general_summary)
        
        # Lab Code Relatled
        graph.add_node("Lab Summary", self.lab_general_summary)
        
        # EMR Note
        graph.add_node("EMR Summary", self.EMR_note_summary)
    

        graph.add_edge(START, "Diagnosis Summary")
        graph.add_edge(START, "Rx Summary")
        graph.add_edge(START, "Procedure Summary")
        graph.add_edge(START, "Lab Summary")
        graph.add_edge(START, "EMR Summary")
        
        graph.add_edge("Diagnosis Summary", END)
        graph.add_edge("Rx Summary", END)
        graph.add_edge("Procedure Summary", END)
        graph.add_edge("Lab Summary", END)
        graph.add_edge("EMR Summary", END)
        
        self.graph = graph.compile(
            checkpointer=checkpointer,
        )
        
        self.tools = {t.name: t for t in tools}
        self.model = model
        self.model_with_tools = model.bind_tools(tools)
        
    def diag_general_summary(self, state: AgentState):
        
        start_time = time.time()
        if len(state["codes_diag"]) == 0:
            return {'messages': [AIMessage(content="No diag codes.")]}
        else:
            message = [
            SystemMessage(content=self.system), 
            HumanMessage(content=DIAG_SUMMARY_PROMPT.format( \
                lookupTable = self.tools['createLookupTable'].invoke({"codes": state['codes_diag'], "lookup_dict": state['lookup_dict_diag']}))) #, \
                # codeHistory = self.tools['createCodeHistory'].invoke({"dates": state['dates_diag'], "codes": state['codes_diag']})))
            ]
            
            print("TIME DIAGNOSIS 1: ", time.time()-start_time)

            # print("Human Message", message)
            response = self.model.invoke(message)
            
            print("TIME DIAGNOSIS 2: ", time.time()-start_time)
        
            # print("-------")
            # print()
            # print("Response", response.content)
            # print()
            # print("-------")
            tempJson = json.loads(response.content.strip('```json').strip('```').strip())
            
            print(tempJson)
            
            # Check for missing codes by LLM, if so we recall the LLM to categorize the missing codes
            incorrectCodes,  missingCodesDict, codesFound, categories = find_missing_codes(tempJson, state['lookup_dict_diag'])
            
            totalMissingCodes = []
            totalMissingCodes.append(missingCodesDict)
            totalIncorrectCodes = []
            totalIncorrectCodes.append(incorrectCodes)
            
            print("TIME DIAGNOSIS 3: ", time.time()-start_time)
            
            count = 0
            while (len(missingCodesDict) > 0 or len(incorrectCodes) > 0) and count < 3:
                
                checkResponse = recall(llm, recall_prompt.format(codeType="diagnosis", categories=categories, codes=missingCodesDict))
                checkJson = json.loads(checkResponse.content.strip('```json').strip('```').strip())
                
                print("Iteration " + str(count) + " # missing codes: ", len(missingCodesDict))
                print("missing codes:", missingCodesDict)
                print("incorrectCodes:", incorrectCodes)
                
                # merge the new recalled categorization from the missing values
                mergedJson = merge_json_data(tempJson[list(tempJson.keys())[0]], checkJson)
                tempJson[list(tempJson.keys())[0]] = mergedJson
                
                # Check again with the new updated tempJson
                incorrectCodes,  missingCodesDict, codesFound, categories = find_missing_codes(tempJson, state['lookup_dict_diag'])
                totalMissingCodes.append(missingCodesDict)
                totalIncorrectCodes.append(incorrectCodes)
                
                count+=1
                
            print("TIME DIAGNOSIS 4: ", time.time()-start_time)

                
            markdownSummary = json_to_markdown(tempJson, state['codes_diag'], state['dates_diag'], state['lookup_dict_diag'], categoryName="diagCategories", missingCodes=totalMissingCodes, incorrectCodes=totalIncorrectCodes, title="# Diagnosis Categories\n")
            
            print("TIME DIAGNOSIS 5: ", time.time()-start_time)
            return {"messages": [AIMessage(content=markdownSummary)]}
    
    def rx_general_summary(self, state: AgentState):
        if len(state["codes_rx"]) == 0:
            return {'messages': [AIMessage(content="No rx codes.")]}
        
        else:
            message = [
            SystemMessage(content=self.system), 
            HumanMessage(content=RX_SUMMARY_PROMPT.format( \
                lookupTable = self.tools['createLookupTable'].invoke({"codes": state['codes_rx'], "lookup_dict": state['lookup_dict_rx']}))) #, \
                # codeHistory = self.tools['createCodeHistory'].invoke({"dates": state['dates_rx'], "codes": state['codes_rx']})))
            ]
            response = self.model.invoke(message)
            
            tempJson = json.loads(response.content.strip('```json').strip('```').strip())
            
            # Check for missing codes by LLM, if so we recall the LLM to categorize the missing codes
            incorrectCodes,  missingCodesDict, codesFound, categories = find_missing_codes(tempJson, state['lookup_dict_rx'])
            
            totalMissingCodes = []
            totalMissingCodes.append(missingCodesDict)
            totalIncorrectCodes = []
            totalIncorrectCodes.append(incorrectCodes)
            
            count = 0
            while (len(missingCodesDict) > 0 or len(incorrectCodes) > 0) and count < 3:
                
                checkResponse = recall(llm, recall_prompt.format(codeType="Rx", categories=categories, codes=missingCodesDict))
                checkJson = json.loads(checkResponse.content.strip('```json').strip('```').strip())
                
                print("Iteration " + str(count) + " # missing codes: ", len(missingCodesDict))
                print("missing codes:", missingCodesDict)
                print("incorrectCodes:", incorrectCodes)
                
                # merge the new recalled categorization from the missing values
                mergedJson = merge_json_data(tempJson[list(tempJson.keys())[0]], checkJson)
                tempJson[list(tempJson.keys())[0]] = mergedJson
                
                # Check again with the new updated tempJson
                incorrectCodes,  missingCodesDict, codesFound, categories = find_missing_codes(tempJson, state['lookup_dict_rx'])
                totalMissingCodes.append(missingCodesDict)
                totalIncorrectCodes.append(incorrectCodes)
                
                count+=1
            
            
            markdownSummary = json_to_markdown(tempJson, state['codes_rx'], state['dates_rx'], state['lookup_dict_rx'], categoryName="rxCategories", missingCodes=totalMissingCodes, incorrectCodes=totalIncorrectCodes, title="# Rx Categories\n")
            return {"messages": [AIMessage(content=markdownSummary)]}
    
    def proc_general_summary(self, state: AgentState):
               
        if len(state["codes_proc"]) == 0:
            return {'messages': [AIMessage(content="No proc codes.")]}
        
        else:
            message = [
            SystemMessage(content=self.system), 
            HumanMessage(content=PROC_SUMMARY_PROMPT.format( \
                lookupTable = self.tools['createLookupTable'].invoke({"codes": state['codes_proc'], "lookup_dict": state['lookup_dict_proc']}))) #, \
                # codeHistory = self.tools['createCodeHistory'].invoke({"dates": state['dates_proc'], "codes": state['codes_proc']})))
            ]
            response = self.model.invoke(message)
            
            tempJson = json.loads(response.content.strip('```json').strip('```').strip())
            
            # Check for missing codes by LLM, if so we recall the LLM to categorize the missing codes
            incorrectCodes,  missingCodesDict, codesFound, categories = find_missing_codes(tempJson, state['lookup_dict_proc'])
            
            totalMissingCodes = []
            totalMissingCodes.append(missingCodesDict)
            totalIncorrectCodes = []
            totalIncorrectCodes.append(incorrectCodes)
            
            count = 0
            while (len(missingCodesDict) > 0 or len(incorrectCodes) > 0) and count < 3:
                
                checkResponse = recall(llm, recall_prompt.format(codeType="Procedure", categories=categories, codes=missingCodesDict))
                
                # print("-------------------")
                # print("Content")
                # print()
                # print(checkResponse.content)
                # print()
                # print()
                # print("-------------------")
                checkJson = json.loads(checkResponse.content.strip('```json').strip('```').strip())
                
                print("Iteration " + str(count) + " # missing codes: ", len(missingCodesDict))
                print("missing codes:", missingCodesDict)
                print("incorrectCodes:", incorrectCodes)
                
                # merge the new recalled categorization from the missing values
                mergedJson = merge_json_data(tempJson[list(tempJson.keys())[0]], checkJson)
                tempJson[list(tempJson.keys())[0]] = mergedJson
                
                # Check again with the new updated tempJson
                incorrectCodes,  missingCodesDict, codesFound, categories = find_missing_codes(tempJson, state['lookup_dict_proc'])
                totalMissingCodes.append(missingCodesDict)
                totalIncorrectCodes.append(incorrectCodes)
                
                count+=1
                
                
            markdownSummary = json_to_markdown(tempJson, state['codes_proc'], state['dates_proc'], state['lookup_dict_proc'], categoryName="procCategories", missingCodes=totalMissingCodes, incorrectCodes=totalIncorrectCodes, title="# Procedure Categories\n")
        
            return {"messages": [AIMessage(content=markdownSummary)]}
    
    def lab_general_summary(self, state: AgentState):
            
        if len(state["codes_lab"]) == 0:
            return {'messages': [AIMessage(content="No lab codes.")]}
        
        else:
            message = [
            SystemMessage(content=self.system), 
            HumanMessage(content=LAB_SUMMARY_PROMPT.format( \
                lookupTable = self.tools['createLabLookupTable'].invoke({"codes": state['codes_lab'], "lookup_dict": state['lookup_dict_lab']}))) #, \
                # codeHistory = self.tools['createLabCodeHistory'].invoke({"dates": state['dates_lab'], "codes": state['codes_lab']}))) 
            ]
            response = self.model.invoke(message)
            # print(response)
            
            tempJson = json.loads(response.content.strip('```json').strip('```').strip())
            
           # Check for missing codes by LLM, if so we recall the LLM to categorize the missing codes
            incorrectCodes,  missingCodesDict, codesFound, categories = find_missing_codes(tempJson, state['lookup_dict_lab'])
            
            totalMissingCodes = []
            totalMissingCodes.append(missingCodesDict)
            totalIncorrectCodes = []
            totalIncorrectCodes.append(incorrectCodes)
            
            count = 0
            while (len(missingCodesDict) > 0 or len(incorrectCodes) > 0) and count < 3:
                
                checkResponse = recall(llm, recall_prompt.format(codeType="Lab", categories=categories, codes=missingCodesDict))
                checkJson = json.loads(checkResponse.content.strip('```json').strip('```').strip())
                
                print("Iteration " + str(count) + " # missing codes: ", len(missingCodesDict))
                print("missing codes:", missingCodesDict)
                print("incorrectCodes:", incorrectCodes)
                
                # merge the new recalled categorization from the missing values
                mergedJson = merge_json_data(tempJson[list(tempJson.keys())[0]], checkJson)
                tempJson[list(tempJson.keys())[0]] = mergedJson
                
                # Check again with the new updated tempJson
                incorrectCodes,  missingCodesDict, codesFound, categories = find_missing_codes(tempJson, state['lookup_dict_lab'])
                totalMissingCodes.append(missingCodesDict)
                totalIncorrectCodes.append(incorrectCodes)
                count+=1
            
            
            markdownSummary = json_to_markdown(tempJson, state['codes_lab'], state['dates_lab'], state['lookup_dict_lab'], categoryName="labCategories", missingCodes=totalMissingCodes, incorrectCodes=totalIncorrectCodes, title="# Lab Categories\n", isLab=True)
            return {"messages": [AIMessage(content=markdownSummary)]}
        
    def EMR_note_summary(self, state: AgentState):
        message = [
        SystemMessage(content=self.system), 
        HumanMessage(content=EMR_NOTE_EXTRACTION.format(EMR=state['EMR_Notes']))
        ]
        response = self.model.invoke(message)
        
        # return {"messages": [response, AIMessage(content=markdownSummary)]}
        return {"messages": [response]}

    # def EMR_HOS_summary(self, state: AgentState):
        
        

In [21]:
# Function to create the desired formatted string
def create_string_from_dataframe(df):
    message = ""
    
    for index, row in df.iterrows():
        message += f"{row['code']} - {row['dos']}\n"
        message += f"{row['value']}\n\n"
        
    return message.strip()

In [22]:
# EMR_Notes=create_string_from_dataframe(emr_rows[emr_rows["mcid"]==609394282])

EMR_SUMMARY_PROMPT = """
Instructions:
1. Given a patient's EMR Notes please output the given format

Format your response like this

Patient demographics:
This includes the patient's name, age, gender, contact information, and insurance.

Medical history:
This includes the patient's past illnesses, surgeries, allergies, family medical history, and social history.

Clinical notes
This includes the healthcare provider's documentation of patient encounters, assessments, diagnoses, and treatment plans.

Medication management:
This includes the medications the patient is taking, as well as any potential drug interactions or allergies.

Laboratory and imaging results:
This includes the patient's test results, radiology reports, and other diagnostic information.

Order management:
This includes the tests, procedures, consultations, and referrals that have been ordered for the patient.

Clinical decision support:
This includes alerts, reminders, and evidence-based guidelines to help healthcare providers make informed decisions. 

EMR Notes Summaries:
Date of service 1
* Emr Note1 title: summary
Date of service 2
* Emr note2 title: summary



Here are the EMR Notes to format
{EMR}
"""

## Past, Present, and Future Conditions

In [23]:
PAST_SUMMARY =  (
"1. Review the outputs from the basic info, diagnosis, procedure, medication, and lab test agents.\n"
"2. Identify and list all past medical conditions based on the patient's medical history.\n"
"3. Prioritize the past conditions based on their significance and impact on the patient's current health status.\n"
"4. Structure the output as follows:\n"
"   a. Start with a 'Prioritized List of Past Medical Conditions'.\n"
"   b. After the list, add the note: 'See below for a summary and detailed analysis of each condition.'\n"
"   c. Follow with a 'Detailed Analysis of Each Past Medical Condition' section.\n"
"5. For each past condition:\n"
"   a. State the condition clearly.\n"
"   b. Provide a detailed timeline including:\n"
"      - Date(s) when the condition was diagnosed\n"
"      - Date(s) of any related procedures\n"
"      - Date(s) and duration of prescribed medications\n"
"      - Date(s) of relevant lab tests\n"
"   c. For each event (diagnosis, procedure, medication, lab test), clearly display:\n"
"      - The exact date (or most specific timeframe if exact date is unavailable)\n"
"      - The associated medical code\n"
"      - The full name or description of the code\n"
"   d. Describe the treatments or procedures undertaken for the condition, with specific dates, codes, and names.\n"
"   e. List medications prescribed, including start dates, duration of treatment, codes, and full names.\n"
"   f. Provide lab test evidence:\n"
"      - Name of the lab test(s) related to the condition\n"
"      - Date(s) when each test was conducted\n"
"      - Results of each test, including numerical values rounded to two decimal places and reference ranges where applicable\n"
"      - Brief interpretation of the test results in the context of the condition\n"
"   g. Discuss the impact of this past condition on the patient's current health status.\n"
"   h. Provide any additional specific evidence from the patient's records that supports this analysis, including dates, codes, and names for all events.\n"
"6. Ensure that all dates, codes, and full names/descriptions for diagnoses, procedures, treatments, medications, and lab tests are clearly stated for each condition.\n"
"7. If exact dates are not available for any event, clearly state this and provide the most specific timeframe possible based on the available data.\n"
"8. For each lab test mentioned, ensure you provide context on how it relates to the condition and what the results (rounded to two decimal places) indicate about the patient's health status at that time.\n\n"
                 
                 
"EXPECTED_OUTPUT:\n"
"A detailed analysis of the patient's past medical conditions, structured as follows:\n"
"1. Prioritized List of Past Medical Conditions\n"
"2. Detailed Analysis of Each Past Medical Condition:\n"
"   - Clear identification of the condition\n"
"   - Detailed timeline including dates, codes, and full names/descriptions for diagnoses, procedures, medication prescriptions, and relevant lab tests\n"
"   - Description of treatments or procedures with specific dates, codes, and names\n"
"   - List of medications with start dates, duration, codes, and full names\n"
"   - Lab test evidence, including:\n"
"     * Name of relevant lab tests\n"
"     * Dates of tests\n"
"     * Test results with numerical values rounded to two decimal places and reference ranges\n"
"     * Brief interpretation of results\n"
"   - Citing EMR note title as evidence for all categories."
"   - Impact on current health status\n"
"   - Any additional specific evidence from patient records, including dates, codes, and names for all events\n"
"Note: All dates should be clearly stated. If exact dates are unavailable, the most specific timeframe possible should be provided. "
"Ensure that the relevance of each lab test to the condition is explained, and the interpretation of results (rounded to two decimal places) is provided in the context of the patient's health status."

)

In [24]:
FUTURE_SUMMARY = (
"1. Review the outputs from the basic info, diagnosis, procedure, medication, lab test, and summary agents, as well as the past and present condition summaries.\n"
"2. Identify potential future medical conditions based on the patient's medical history, current status, and risk factors.\n"
"3. Exclude any conditions that have already occurred in the patient's past medical history.\n"
"4. Prioritize the potential future conditions based on their likelihood and potential impact on the patient's health.\n"
"5. Structure the output as follows:\n"
"   a. Start with a 'Prioritized List of Potential Future Conditions'.\n"
"   b. After the list, add the note: 'See below for a summary and detailed analysis of each condition.'\n"
"   c. Follow with a 'Detailed Analysis of Each Potential Future Condition' section.\n"
"6. For each potential future condition in the detailed analysis:\n"
"   a. State the potential condition clearly.\n"
"   b. Explain the risk factors present in the patient's history that suggest this potential condition.\n"
"   c. Provide specific evidence from the patient's records that supports this prediction:\n"
"      - For any medications or procedures related to the possibility of the condition, mention:\n"
"        * The name of the medication or procedure\n"
"        * The specific date when it occurred\n"
"        * The associated medical code\n"
"        * The full description of the code\n"
"      - For ANY lab test mentioned:\n"
"        * ALWAYS include the normal range for that test\n"
"        * Round all lab test results to two decimal places\n"
"        * Include the date when the test was conducted\n"
"   d. When mentioning any medical code (diagnosis, procedure, medication), always display:\n"
"      * The proper name of the code\n"
"      * The code description\n"
"      * The date when it occurred\n"
"   e. Estimate the likelihood or timeframe for this condition to develop, if possible.\n"
"   f. Suggest preventive measures or monitoring strategies to mitigate this future health risk.\n\n"

"EXPECTED_OUTPUT:\n"
"A detailed analysis of the patient's potential future medical conditions, structured as follows:\n"
"1. Prioritized List of Potential Future Conditions\n"
"2. Detailed Analysis of Each Potential Future Condition, including for each:\n"
"   - Clear identification of the condition\n"
"   - Explanation of risk factors\n"
"   - Specific evidence supporting the prediction, including:\n"
"     * For diagnosis, medications, and procedures: name, date of occurrence, medical code, and full code description\n"
"     * For ALL lab tests: normal ranges, results rounded to two decimal places, and test dates\n"
"     * Cite EMR notes with their titles if they provide evidence for any category"
"   - For any medical code mentioned: proper name, description, and occurrence date\n"
"   - Estimated likelihood or timeframe for condition development, if possible\n"
"   - Suggested preventive measures or monitoring strategies"
) 

In [25]:
class ConditionsAgent():
    def __init__(self, model, tools, checkpointer, system=""):
        self.system = system
        graph = StateGraph(AgentState)
        
        # Summaries
        graph.add_node("Past Conditions Summary", self.past_conditions)
        graph.add_node("Future Conditions Summary", self.future_conditions)
    
        graph.add_edge(START, "Past Conditions Summary")
        graph.add_edge(START, "Future Conditions Summary")
        
        graph.add_edge("Past Conditions Summary", END)
        graph.add_edge("Future Conditions Summary", END)
        
        self.graph = graph.compile(
            checkpointer=checkpointer,
            #interrupt_before=["User Q and A"]
        )
        
        self.tools = {t.name: t for t in tools}
        self.model = model
        self.model_with_tools = model.bind_tools(tools)
        
    # Gives the common disease summary based on the GENERAL_SUMMARY_PROMPT using the createLookupTable and createCodeHistory tools
    def past_conditions(self, state: AgentState):
        print(state['messages'])
        
        temp = []        
        for i, elem in enumerate(state['messages']):
            print("I index", i)
            print(elem)
            
            
        message = [SystemMessage(content=self.system)] + state['messages'] + [HumanMessage(content=PAST_SUMMARY)]
        response = self.model.invoke(message)
        return {"messages": [response]}
    
    # Gives the common disease summary based on the GENERAL_SUMMARY_PROMPT using the createLookupTable and createCodeHistory tools
    def future_conditions(self, state: AgentState):
        message = [SystemMessage(content=self.system)] + state['messages'] + [HumanMessage(content=FUTURE_SUMMARY)]
        response = self.model.invoke(message)
        return {"messages": [response]}
    

## User Chat Agent Graph

In [26]:
class ChatAgent():
    def __init__(self, model, tools, checkpointer, system=""):
        self.system = system
        graph = StateGraph(AgentState)
        graph.add_node("LLM Answer", self.call_openai)
        graph.add_node("Tool Call", self.take_action)

        #graph.set_entry_point("General Summary")
        graph.add_edge(START, "LLM Answer")
        graph.add_conditional_edges("LLM Answer", self.exists_action, {True: "Tool Call", False: END})
        graph.add_edge("Tool Call", "LLM Answer")
        
        self.graph = graph.compile(
            checkpointer=checkpointer,
            #interrupt_before=["User Q and A"]
        )
        self.tools = {t.name: t for t in tools}
        self.model = model.bind_tools(tools)
        
    def exists_action(self, state: AgentState):
        result = state['messages'][-1]
        return len(result.tool_calls) > 0

    def call_openai(self, state: AgentState):
        messages = state['messages']
        if self.system:
            messages = [SystemMessage(content=self.system)] + messages
        print(messages)
        message = self.model.invoke(messages)
        return {'messages': [message]}

    def take_action(self, state: AgentState):
        tool_calls = state['messages'][-1].tool_calls
        results = []
        images = []
        for t in tool_calls:
            print(f"Calling: {t}")
            if not t['name'] in self.tools:      # check for bad tool name from LLM
                print("\n ....bad tool name....")
                result = "bad tool name, retry"  # instruct LLM to retry if bad
            else:
                if t['name'] == "labViz":
                    #adjust = t['args']
                    #adjust['dates'] = state['dates_lab']
                    #adjust['lab_values'] = state['codes_lab']
                    labVizArgs = self.tools["labViz"].invoke(t['args'])
                    print("hello")
                    print(labVizArgs)
                    result = labVizData(*labVizArgs, state['dates_lab'], state['codes_lab'])
                    print(result)
                else:
                    result = self.tools[t['name']].invoke(t['args'])
                # if isinstance(result, PIL_Image.Image):
                #     images.append(result)
            
            results.append(ToolMessage(tool_call_id=t['id'], name=t['name'], content=str(result)))
        print("Back to the model!")
        return {'messages': results} #, 'images': images}
    
    

In [27]:
from collections import defaultdict
import json
def merge_json_data(json1, json2):
    merged_data = defaultdict(lambda: [])

    # Helper function to process each JSON list
    def add_to_merged(json_data):
        for category in json_data:
            merged_data[category['category']].extend(category['values'])
    
    # Add data from both JSON lists
    add_to_merged(json1)
    add_to_merged(json2)
    
    # Remove duplicates within each category
    final_data = []
    for category, values in merged_data.items():
        unique_values = list({v['code']: v for v in values}.values())
        final_data.append({'category': category, 'values': unique_values})
    
    return final_data

In [28]:
import json

# This function will return codes that were missing 
def find_missing_codes(jsonData, codeDict):
    incorrect_codes = []
    codesFound = set()
    categories = []
     
        
    

        
    for category in jsonData[list(jsonData.keys())[0]]:
        categories.append(category["category"])
        for value in category["values"]:
            codesFound.add(value["code"])
            if value["code"] not in codeDict:
                incorrect_codes.append(value["code"])
            
    missing_codes = {}
    
    for code in codeDict:
        if code not in codesFound:
            missing_codes[code]=codeDict[code]
    
    
    
    return incorrect_codes,  missing_codes, codesFound, categories


In [29]:
recall_prompt = """
Instructions:
1. Look at the provided codes and categories for {codeType} codes
2. Categorize the codes under the existing categories or create new categories to put the codes under
3. Ensure that the codes under each category makes the most sense
4. Keep the existing categories the same order in the output and include every category even if no provided code falls under it
5. If you insert new categories with the following codes, new categories can be inserted anywhere within the current order as long
as the prioritization makes sense in terms of seriousness of the category and conditions, more serious conditions and categories being closer to the top
4. Give your output in json following the provided format

Output Format Json:
[
{{
  "category": "Category Name 1",
  "values": [
    {{
      "name": "code Name 1",
      "code": "Code1",
    }},
    {{
      "name": "code Name 2",
      "code": "Code2"
    }}
  ]
}},
{{
  "category": "Category Name 2",
  "values": [
    {{
      "name": "code Name 3",
      "code": "labCode3"
    }},
    {{
      "name": "code Name 4",
      "code": "Code4"
    }}
  ]
}}
# Continue with additional categories and values as necessary
]


Given these categories:
{categories}

Given these codes:
{codes}
    
"""

def recall(llm, recallPrompt):
    response = llm.invoke(recallPrompt)
    return response
    
    
        

In [30]:
# Function to convert JSON to Markdown format and include medCode and medDate, jsonData is the json
def json_to_markdown(jsonData, medCode, medDate, medDict, categoryName, missingCodes, incorrectCodes, title="# med Categories\n", isLab=False):

    
    markdown_string = title
    markdown_string += f"Missing Codes: {missingCodes}, Incorrect Codes: {incorrectCodes}\n"
    
    
    if isLab == True:
        for category in jsonData[categoryName]:
            markdown_string += f"## {category['category']}\n"

            for value in category['values']:
                # markdown_string += f"- **Name:** {value['name']}\n"
                # markdown_string += f"  - **Code:** {value['code']}\n"
                # markdown_string += f"- **{value['name']}** ({value['code']})\n"
                
                markdown_string += f"- **{medDict[value['code']]}** ({value['code']})\n"
                
                # Add dates for the specific code
                associated_dates = []
                for codes, date in zip(medCode, medDate):
                    if value['code'] in codes:
                        associated_dates.append(date + ": " + "**"+str(codes[value['code']][0]) + " " + str(codes[value['code']][1]) + "**")
                if associated_dates:
                    markdown_string += f"  - {', '.join(associated_dates)}\n"
                    

            markdown_string += "\n"
    else:
        for category in jsonData[categoryName]:
            markdown_string += f"## {category['category']}\n"

            for value in category['values']:
                # markdown_string += f"- **Name:** {value['name']}\n"
                # markdown_string += f"  - **Code:** {value['code']}\n"
                # markdown_string += f"- **{value['name']}** ({value['code']})\n"
                
                markdown_string += f"- **{medDict[value['code']]}** ({value['code']})\n"

                # Add dates for the specific code
                associated_dates = []
                for codes, date in zip(medCode, medDate):
                    if value['code'] in codes:
                        associated_dates.append(date)
                if associated_dates:
                    markdown_string += f"  - {', '.join(associated_dates)}\n"

            markdown_string += "\n"
    
    return markdown_string.strip()


In [31]:
def getDatesCodesList(mcid, inputDf, typeCode = 'diag', gradio=False):
    
    if gradio == True:
        inputDf = inputDf[inputDf["id"] == mcid]
    else:
        inputDf = inputDf[inputDf["mcid"] == mcid]
    
    # Filter for 'diag' codes
    code_df = inputDf[inputDf['type'] == typeCode]
    
    codeDict = dict(zip(code_df['code'], code_df['codeDesc']))

    if typeCode == 'lab':
        # If you want a list of lists with unique dates
        unique_dates = code_df['dos'].unique()
        
        # Create a list of dictionaries corresponding to each unique date
        codes_by_date = [
            # Making this a tuple of type (lab value, abnormal value)
            {row['code']: (str(round(float(row['value']), 3)), row['abnormal_cd']) for idx, row in code_df[code_df['dos'] == date].iterrows()}
            for date in unique_dates
        ]

    else:
        # If you want a list of lists with unique dates
        unique_dates = code_df['dos'].unique()
        codes_by_date = [code_df[code_df['dos'] == date]['code'].tolist() for date in unique_dates]
    
    
    
    
    return list(unique_dates), list(codes_by_date), codeDict

## Explore Member EMR

In [32]:
def check_conditions(emr_df, code_df, mcid_list):
    # Filter `emr_rows` by `mcidList`
    emr_filtered = emr_df[emr_df["mcid"].isin(mcid_list)]
    
    # Check which mcids exist in `emr_rows` after filtering
    mcid_present = emr_filtered["mcid"].unique()
    
    # Filter `code_rows` for relevant `mcid`s
    code_filtered = code_df[code_df["mcid"].isin(mcid_present)]
    
    # Create a pivot table to count occurrences of types per `mcid`
    code_pivot = code_filtered.pivot_table(index='mcid', columns='type', aggfunc='size', fill_value=0)
    
    # Check for the required types
    required_types = ["proc", "diag", "lab", "rx"]
    
    # Ensure that each `mcid` has at least one of each required type
    valid_mcids = code_pivot.index[(code_pivot[required_types] > 20).all(axis=1)]
    
    return valid_mcids.tolist()

# Run the function to get the list of `mcid`s meeting all conditions
notMissingList = check_conditions(emr_rows, code_rows, mcidList)

In [33]:
len(notMissingList)

648

In [34]:
longNotMissingList = []
for i in notMissingList:
    if len(create_string_from_dataframe(emr_rows[emr_rows["mcid"]==i])) > 90000:
        longNotMissingList.append(i)

In [35]:
len(longNotMissingList)

44

In [36]:
longNotMissingList

[9872029,
 9874073,
 9897234,
 9912281,
 9941873,
 19728651,
 25344889,
 25424957,
 27517922,
 28333344,
 28945980,
 28947646,
 28960230,
 28993194,
 28994102,
 28994158,
 29008273,
 29027524,
 29058141,
 29070586,
 33869591,
 62391707,
 105458391,
 202143893,
 210535043,
 210542247,
 211986176,
 212792175,
 321664611,
 322307442,
 395804208,
 405818055,
 407750262,
 407945635,
 415304200,
 434168628,
 436963396,
 473816741,
 479962938,
 604786481,
 617114373,
 631472069,
 645319686,
 744034569]

In [37]:
start_summ = { "lookup_dict_diag": diagDict, "dates_diag": diagDate, "codes_diag": diagCode, \
              "lookup_dict_lab": labDict, "dates_lab": labDate, "codes_lab": labCode, \
              "lookup_dict_rx": rxDict, "dates_rx": rxDate, "codes_rx": rxCode, \
              "lookup_dict_proc": procDict, "dates_proc": procDate, "codes_proc": procCode, \
              "verbose": 1}

NameError: name 'diagDict' is not defined

In [ ]:
thread = {"configurable": {"thread_id": "1"}}
events_summ = []

start_time = time.time()
for event in summAgent.graph.stream(start_summ, thread):
    for v in event.values():
        events_summ.append(v)
        print(v)
        
end_time = time.time() - start_time
print("Runtime:", end_time)


In [ ]:
print(create_string_from_dataframe(emr_rows[emr_rows["mcid"]==mcidList[1]]))

In [ ]:
display(Markdown(events_summ[4]['messages'][0].content))

In [ ]:
prompt = """You are a smart patient medical codes analyzer. Give a good summary, prioritizing more important information to the top, \
for each type of medical history provided for a patient."""
llm = ChatOpenAI(model="gpt-4o-0806", temperature=0)
conditAgent = ConditionsAgent(llm, [tavily], checkpointer=memory, system=prompt)

# from IPython.display import Image
# Image(conditAgent.graph.get_graph().draw_png())

In [ ]:
thread = {"configurable": {"thread_id": "1"}}
condit_summ = []

# "messages": [events_summ[0]['messages'][1] ]

start_time = time.time()
for event in conditAgent.graph.stream({"verbose": 1}, thread):
    for v in event.values():
        condit_summ.append(v)
        print(v)
        
end_time = time.time() - start_time
print("Runtime:", end_time)


In [ ]:
display(Markdown(condit_summ[1]['messages'][0].content))

## Helper functions for loading images

In [ ]:
import re

def find_image_filenames(text):
    """Find all image filenames in the given text."""
    return re.findall(r'!\[.*?\]\((.*?)\)', text)


In [ ]:
from PIL import Image as PIL_Image

def load_images(image_filenames):
    """Load images whose filenames are in the given list."""
    return [PIL_Image.open(fname) for fname in image_filenames]


In [ ]:
def pil_images_to_base64(image_filenames):
    """
    Converts a list of PIL images to a list of base64-encoded strings.
    
    Args:
    images (list): List of PIL images.
    
    Returns:
    list: List of base64-encoded strings.
    """
    images = load_images(image_filenames)
    
    img_str_list = []
    
    for i, image in enumerate(images):
        buffer = io.BytesIO()
        
        image.save(buffer, format="PNG")
        buffer.seek(0)
        
        img_str = base64.b64encode(buffer.read()).decode('utf-8')
        markdown_img_str = f"![{image_filenames[i]}](data:image/jpeg;base64,{img_str})"
        
        img_str_list.append(markdown_img_str)
        buffer.close()
        
    return img_str_list

In [ ]:
import re

def replace_image_markdown(text, new_strings):
    """Replace all image markdowns in the given text with the provided strings."""
    # This is the function that `sub()` will use. It replaces each match with the next string in new_strings
    replace_fn = lambda match: new_strings.pop(0) if new_strings else match.group(0)
    
    # In the text, replace each image markdown using replace_fn
    new_text = re.sub(r'!\[.*?\]\(.*?\)', replace_fn, text)
    
    return new_text


# Demo: Frontend

In [ ]:
EMR_SUMMARY_PROMPT = """
Instructions:
1. Given a patient's EMR Notes please output the given format

Format your response like this

Patient demographics:
This includes the patient's name, age, gender, contact information, and insurance.

Medical history:
This includes the patient's past illnesses, surgeries, allergies, family medical history, and social history.

Clinical notes
This includes the healthcare provider's documentation of patient encounters, assessments, diagnoses, and treatment plans.

Medication management:
This includes the medications the patient is taking, as well as any potential drug interactions or allergies.

Laboratory and imaging results:
This includes the patient's test results, radiology reports, and other diagnostic information.

Order management:
This includes the tests, procedures, consultations, and referrals that have been ordered for the patient.

Clinical decision support:
This includes alerts, reminders, and evidence-based guidelines to help healthcare providers make informed decisions. 

EMR Notes Summaries:
Date of service 1
* Emr Note1 title: summary
Date of service 2
* Emr note2 title: summary

Here are the EMR Notes to format
{EMR}
"""

In [ ]:
EMR_NOTE_EXTRACTION = (
"1. Review the patient's EMR notes.\n"
"2. Identify and extract the following key pieces of information:\n"
"   a. Current Medical Conditions\n"
"   b. Medications and Dosages\n"
"   c. Recent Procedures\n"
"   d. Lab Test Results\n"
"   e. Recommendations or Follow-Up Instructions\n"
"3. Prioritize the extracted information based on its relevance to the patient's current health status.\n"
"4. Structure the output as follows:\n"
"   a. Start with a 'Summary of Current Medical Findings'.\n"
"   b. Follow with a 'Detailed Analysis of Each Key Finding' section.\n"
"5. For each key piece of information, provide:\n"
"   a. A clear identification of the finding (e.g., name of the condition, medication, procedure, or lab test).\n"
"   b. Relevant details including:\n"
"      - Dates when the information was recorded or last updated\n"
"      - Specifics and context from the EMR notes\n"
"6. For Current Medical Conditions:\n"
"   a. State the condition clearly.\n"
"   b. Provide a brief description of the condition based on the notes.\n"
"   c. Include any relevant dates and details of diagnosis or observation.\n"
"7. For Medications and Dosages:\n"
"   a. List each medication including its name and dosage.\n"
"   b. Provide the start date, duration, and any special instructions as mentioned in the notes.\n"
"8. For Recent Procedures:\n"
"   a. Describe the procedure clearly with its name.\n"
"   b. Include the date it was performed and any pertinent notes or outcomes.\n"
"9. For Lab Test Results:\n"
"   a. Provide the name of each lab test.\n"
"   b. Include the date when the test was conducted.\n"
"   c. Display the results with numerical values rounded to two decimal places and reference ranges where applicable.\n"
"   d. Offer a brief interpretation of the results in the context of the patient's health.\n"
"10. For Recommendations or Follow-Up Instructions:\n"
"   a. Clearly state any recommendations or instructions given to the patient.\n"
"   b. Include any specific dates, times, or actions the patient needs to take.\n"
"11. Ensure that all dates, names, and descriptions are clearly stated for each piece of information.\n"
"12. If exact dates are not available, clearly state this and provide the most specific timeframe possible based on the available notes.\n\n"

"EXPECTED_OUTPUT:\n"
"A comprehensive summary and detailed analysis of the patient's current health status based on EMR notes, structured as follows:\n"
"1. Summary of Current Medical Findings\n"
"2. Detailed Analysis of Each Key Finding:\n"
"   - Clear identification of the finding\n"
"   - Relevant details including dates and specific context from the notes\n"
"3. For Current Medical Conditions:\n"
"   - Name of the condition\n"
"   - Brief description and relevant dates of diagnosis or observation\n"
"4. For Medications and Dosages:\n"
"   - Name and dosage of each medication\n"
"   - Start date, duration, and any special instructions\n"
"5. For Recent Procedures:\n"
"   - Name and date of each procedure\n"
"   - Pertinent notes or outcomes\n"
"6. For Lab Test Results:\n"
"   - Name and date of each lab test\n"
"   - Results with numerical values and reference ranges\n"
"   - Brief interpretation of results\n"
"7. For Recommendations or Follow-Up Instructions:\n"
"   - Clear statement of recommendations or instructions\n"
"   - Specific dates, times, or actions required of the patient\n"
"Note: All dates should be clearly stated. If exact dates are unavailable, the most specific timeframe possible should be provided."
"Ensure that the details extracted are accurately and clearly presented to the patient, enhancing their understanding of their current health status and necessary actions.\n\n"    
"EMR Notes information: {EMR}"
)


In [ ]:
longNotMissingList

In [ ]:
emr_rows[emr_rows["mcid"].isin(longNotMissingList)]

In [ ]:
import functools

memory_chat = SqliteSaver.from_conn_string(":memory:")
total_events_summ = []
total_patient_info = []

def getSummaries(id_value=0, thread_id = "1"):
    global memory_chat
    global total_events_summ
    global total_patient_info
    
    EMR_Notes = create_string_from_dataframe(emr_rows[emr_rows["id"]==id_value])
    
    
    age = gender_rows.query(f'id == {id_value} and type == "age"')['value'].iloc[0]
    gender = gender_rows.query(f'id == {id_value} and type == "gender"')['value'].iloc[0]
    
    
    if gender == "1.0":
        gender = "male"
    else:
        gender = "female"
    
    total_patient_info.append("Age: " + age + "\n\n" + " Gender: " + gender)
    
    # Selecting the Patient
    patient = []
    
    
    llm = ChatOpenAI(model="gpt-4o-0806", temperature=0)

    prompt = """You are a smart patient medical codes analyzer. Give a good summary, prioritizing more important information to the top, \
for each type of medical history provided for a patient."""

    ## SUMMARY AGENT
    summAgent = SummaryAgent(llm, [createLookupTable, createCodeHistory, createLabLookupTable, createLabCodeHistory, returnSpecificICD10DatesCodes], checkpointer=memory_chat, system=prompt)
    
    diagDate, diagCode, diagDict = getDatesCodesList(id_value, code_rows, 'diag', gradio=True)
    rxDate, rxCode, rxDict = getDatesCodesList(id_value, code_rows, 'rx', gradio=True)
    procDate, procCode, procDict = getDatesCodesList(id_value, code_rows, 'proc', gradio=True)
    labDate, labCode, labDict = getDatesCodesList(id_value, code_rows, 'lab', gradio=True)
    
    start_summ = {"lookup_dict_diag": diagDict, "dates_diag": diagDate, "codes_diag": diagCode, \
                  "lookup_dict_lab": labDict, "dates_lab": labDate, "codes_lab": labCode, \
                  "lookup_dict_rx": rxDict, "dates_rx": rxDate, "codes_rx": rxCode, \
                  "lookup_dict_proc": procDict, "dates_proc": procDate, "codes_proc": procCode, \
                  "verbose": 1, "EMR_Notes": EMR_Notes}

    thread = {"configurable": {"thread_id": thread_id}}
    events_summ = []

    
    start_time = time.time()
    for event in summAgent.graph.stream(start_summ, thread):
        for v in event.values():
            events_summ.append(v)
            print(v)
            
    end_time = time.time() - start_time
    total_events_summ.append(events_summ)
    print("Runtime:", end_time)
    
    ## CONDITIONS AGENT


    prompt = """You are a smart patient medical codes analyzer. Give a good summary, prioritizing more important information to the top, \
for each type of medical history provided for a patient."""

    llm = ChatOpenAI(model="gpt-4o-0806", temperature=0)

    conditAgent = ConditionsAgent(llm, [tavily], checkpointer=memory_chat, system=prompt)

    thread = {"configurable": {"thread_id": thread_id}}
    condit_summ = []

    start_time = time.time()
    for event in conditAgent.graph.stream({"verbose": 1}, thread):
        for v in event.values():
            condit_summ.append(v)
            print(v)

    end_time = time.time() - start_time
    total_events_summ[-1].extend(condit_summ)
    print("Runtime:", end_time)
    

def getLoadedSummaries(mcid=0, thread_id = "1"):
    global memory_chat
    global total_events_summ
    global total_patient_info
    
    start_time = time.time()
    total_patient_info.append("Age: " + str(df1.iloc[mcid]['num_age']) + " Gender Male: " + str(df1.iloc[mcid]['num_genderm']))
    
    with open('data.json', 'r') as file:
        data = json.load(file)
    
    loaded_patient = None
    for i, elem in enumerate(data):
        if elem[0] == mcid:
            loaded_patient = elem

    print(loaded_patient[1:])
    end_time = time.time() - start_time
    total_events_summ.append(loaded_patient[1:])
    print("Runtime:", end_time)

    
current_tab = 0
mcids = [None, None, None, None, None]  

prompt = """You are a smart medical expert, well versed in medical codes. Use the search engine to look up information. \
Use the lab visualization tool to save a graphed image of a specific lab code (remember to use the exact lab_code name). Graphed image name should be this format ![An example lab image](example_lab.png), and written where it should be displayed. \
You are allowed to make multiple calls (either together or in sequence). \
Only look up information when you are sure of what you want. \
If you need to look up some information before asking a follow up question, you are allowed to do that! \
Also, insert the graph where it makes sense for the answer to the user's question!"""

llm = ChatOpenAI(model="gpt-4o-0806", temperature=0)

x = ChatAgent(llm, [tavily, labViz], checkpointer=memory_chat, system=prompt)

def show_next_tab(user_input):
    ## Creates next tab
    global current_tab
    value = int(user_input)
    updates = [gr.update(visible=True) if i < current_tab else gr.update(visible=False) for i in range(5)]
    if current_tab < 5:
        current_tab += 1
        mcids[current_tab-1] = value
        print(mcids)
        print("thread_id:", current_tab)
        updates = [gr.update(visible=True) if i < current_tab else gr.update(visible=False) for i in range(5)]

        getSummaries(id_value=mcids[current_tab-1], thread_id=str(current_tab))
            
    return updates

def show_next_tab_load(user_input):
    ## Creates next tab
    global current_tab
    value = int(user_input)
    updates = [gr.update(visible=True) if i < current_tab else gr.update(visible=False) for i in range(5)]
    
    with open('data.json', 'r') as file:
        data = json.load(file)
    
    
    saved_mcids = []
    for i, elem in enumerate(data):
        saved_mcids.append(elem[0])
    
    if value in saved_mcids:
        if current_tab < 5:
            current_tab += 1
            mcids[current_tab-1] = value
            print(mcids)
            print("thread_id:", current_tab)
            updates = [gr.update(visible=True) if i < current_tab else gr.update(visible=False) for i in range(5)]
            
            getLoadedSummaries(mcid=mcids[current_tab-1], thread_id=str(current_tab))
    return updates

    
css = """
h1 {
    text-align: center;
    display:block;
}
"""

with gr.Blocks(fill_height=True, fill_width=True,css=css, theme=gr.themes.Soft(primary_hue=gr.themes.colors.blue, secondary_hue=gr.themes.colors.sky)) as demo:
    # page_num = gr.Number(value=0, label="Page Number")
    gr.Markdown("# Chat with Medical Records")
    def respond(message, chat_history, thread_id="1"):
        thread = {"configurable": {"thread_id": thread_id}}
        messages = [HumanMessage(content=message)]
        events_user = []
        for event in x.graph.stream({"messages": messages, "verbose": 1}, thread):
            for v in event.values():
                events_user.append(v)

        image_names = find_image_filenames(events_user[-1]['messages'][0].content)
        loaded_base64 = pil_images_to_base64(image_names)
        final_message = replace_image_markdown(events_user[-1]['messages'][0].content, loaded_base64)
        chat_history.append((message , final_message))
        return "", chat_history
    
    def update_content(page_num=1):
        global total_events_summ
        if isinstance(total_events_summ[page_num-1][0], str):
            return (total_events_summ[page_num-1][0], 
                    total_events_summ[page_num-1][1], 
                    total_events_summ[page_num-1][2], 
                    total_events_summ[page_num-1][3], 
                    total_events_summ[page_num-1][4],
                    total_events_summ[page_num-1][5],
                    total_events_summ[page_num-1][6],
                    total_patient_info[page_num-1])
        else:
            return (total_events_summ[page_num-1][0]['messages'][0].content, 
                    total_events_summ[page_num-1][1]['messages'][0].content, 
                    total_events_summ[page_num-1][2]['messages'][0].content, 
                    total_events_summ[page_num-1][3]['messages'][0].content, 
                    total_events_summ[page_num-1][4]['messages'][0].content,
                    total_events_summ[page_num-1][5]['messages'][0].content,
                    total_events_summ[page_num-1][6]['messages'][0].content,
                    total_patient_info[page_num-1])
    
    def hide_content():
        return ("", "", "", "", "", "", "", "", "")
    
    with gr.Tabs() as tabs:
        with gr.Tab('Patient 1', visible = False, ) as tab1:
            with gr.Column():
                with gr.Tabs() as pageTabs1:
                    with gr.Tab('Diagnosis Summary', visible = True) as diagPage1:
                        contentDiag1 = gr.Markdown()
                    with gr.Tab('Rx Summary', visible = True) as rxPage1:
                        contentRx1 = gr.Markdown()
                    with gr.Tab('Procedure Summary', visible = True) as procPage1:
                        contentProc1 = gr.Markdown()
                    with gr.Tab('Lab Summary', visible = True) as labPage1:
                        contentLab1 = gr.Markdown()
                    with gr.Tab('EMR Summary', visible = True) as emrPage1:
                        contentEMR1 = gr.Markdown()
                    with gr.Tab('Previous Conditions', visible = True) as prevConditPage1:
                        contentPrevCondit1 = gr.Markdown()
                    with gr.Tab('Future Conditions to Watch', visible = True) as futConditPage1:
                        contentFutCondit1 = gr.Markdown()
                    with gr.Tab('Patient Information', visible = True) as patientInfoPage1:
                        contentPatientInfo1 = gr.Markdown()

                    show_summaries_button = gr.Button("Show Summaries")
                    show_summaries_button.click(fn=update_content, inputs=None, outputs=[contentDiag1, contentRx1, contentProc1, contentLab1, contentEMR1, contentPrevCondit1, contentFutCondit1, contentPatientInfo1])
                    
                    hide_summaries_button = gr.Button("Hide Summaries")
                    hide_summaries_button.click(fn=hide_content, inputs=None, outputs=[contentDiag1, contentRx1, contentProc1, contentLab1, contentEMR1, contentPrevCondit1, contentFutCondit1, contentPatientInfo1])
                           
            with gr.Column():
                patient1_chat = gr.Chatbot(label="patient1_chatbot", height=800)
                with gr.Row():
                    text_input = gr.Textbox(label="Your Message")
                    send_button = gr.Button("Send")
                send_button.click(fn=respond, inputs=[text_input, patient1_chat], outputs=[text_input, patient1_chat])

        with gr.Tab('Patient 2', visible = False, ) as tab2:
            with gr.Column():
                with gr.Tabs() as pageTabs2:
                    with gr.Tab('Diagnosis Summary', visible = True) as diagPage2:
                        contentDiag2 = gr.Markdown()
                    with gr.Tab('Rx Summary', visible = True) as rxPage2:
                        contentRx2 = gr.Markdown()
                    with gr.Tab('Procedure Summary', visible = True) as procPage2:
                        contentProc2 = gr.Markdown()
                    with gr.Tab('Lab Summary', visible = True) as labPage2:
                        contentLab2 = gr.Markdown()
                    with gr.Tab('EMR Summary', visible = True) as emrPage2:
                        contentEMR2 = gr.Markdown()
                    with gr.Tab('Previous Conditions', visible = True) as prevConditPage2:
                        contentPrevCondit2 = gr.Markdown()
                    with gr.Tab('Future Conditions to Watch', visible = True) as futConditPage2:
                        contentFutCondit2 = gr.Markdown()
                    with gr.Tab('Patient Information', visible = True) as patientInfoPage2:
                        contentPatientInfo2 = gr.Markdown()

                    update_content2 = functools.partial(update_content, page_num=2)
                    show_summaries_button = gr.Button("Show Summaries")
                    show_summaries_button.click(fn=update_content2, inputs=None, outputs=[contentDiag2, contentRx2, contentProc2, contentLab2, contentEMR2, contentPrevCondit2, contentFutCondit2, contentPatientInfo2])
                    
                    hide_summaries_button = gr.Button("Hide Summaries")
                    hide_summaries_button.click(fn=hide_content, inputs=None, outputs=[contentDiag2, contentRx2, contentProc2, contentLab2, contentEMR2, contentPrevCondit2, contentFutCondit2, contentPatientInfo2])
                           
            with gr.Column():
                patient2_chat = gr.Chatbot(label="patient2_chatbot", height=800)
                with gr.Row():
                    text_input = gr.Textbox(label="Your Message")
                    send_button = gr.Button("Send")
                respond2 = functools.partial(respond, thread_id="2")
                send_button.click(fn=respond2, inputs=[text_input, patient2_chat], outputs=[text_input, patient2_chat])
            
            
        with gr.Tab('Patient 3', visible = False, ) as tab3:
            with gr.Column():
                with gr.Tabs() as pageTabs3:
                    with gr.Tab('Diagnosis Summary', visible = True) as diagPage3:
                        contentDiag3 = gr.Markdown()
                    with gr.Tab('Rx Summary', visible = True) as rxPage3:
                        contentRx3 = gr.Markdown()
                    with gr.Tab('Procedure Summary', visible = True) as procPage3:
                        contentProc3 = gr.Markdown()
                    with gr.Tab('Lab Summary', visible = True) as labPage3:
                        contentLab3 = gr.Markdown()
                    with gr.Tab('EMR Summary', visible = True) as emrPage3:
                        contentEMR3 = gr.Markdown()
                    with gr.Tab('Previous Conditions', visible = True) as prevConditPage3:
                        contentPrevCondit3 = gr.Markdown()
                    with gr.Tab('Future Conditions to Watch', visible = True) as futConditPage3:
                        contentFutCondit3 = gr.Markdown()
                    with gr.Tab('Patient Information', visible = True) as patientInfoPage3:
                        contentPatientInfo3 = gr.Markdown()

                    
                    update_content3 = functools.partial(update_content, page_num=3)
                    show_summaries_button = gr.Button("Show Summaries")
                    show_summaries_button.click(fn=update_content3, inputs=None, outputs=[contentDiag3, contentRx3, contentProc3, contentLab3, contentEMR3, contentPrevCondit3, contentFutCondit3, contentPatientInfo3])
                    
                    hide_summaries_button = gr.Button("Hide Summaries")
                    hide_summaries_button.click(fn=hide_content, inputs=None, outputs=[contentDiag3, contentRx3, contentProc3, contentLab3, contentEMR3, contentPrevCondit3, contentFutCondit3, contentPatientInfo3])
                           
            with gr.Column():
                patient3_chat = gr.Chatbot(label="patient3_chatbot", height=800)
                with gr.Row():
                    text_input = gr.Textbox(label="Your Message")
                    send_button = gr.Button("Send")
                respond3 = functools.partial(respond, thread_id="3")
                send_button.click(fn=respond3, inputs=[text_input, patient3_chat], outputs=[text_input, patient3_chat])

                   
        with gr.Tab('Patient 4', visible = False, ) as tab4:
            with gr.Column():
                with gr.Tabs() as pageTabs4:
                    with gr.Tab('Diagnosis Summary', visible = True) as diagPage4:
                        contentDiag4 = gr.Markdown()
                    with gr.Tab('Rx Summary', visible = True) as rxPage4:
                        contentRx4 = gr.Markdown()
                    with gr.Tab('Procedure Summary', visible = True) as procPage4:
                        contentProc4 = gr.Markdown()
                    with gr.Tab('Lab Summary', visible = True) as labPage4:
                        contentLab4 = gr.Markdown()
                    with gr.Tab('EMR Summary', visible = True) as emrPage4:
                        contentEMR4 = gr.Markdown()
                    with gr.Tab('Previous Conditions', visible = True) as prevConditPage4:
                        contentPrevCondit4 = gr.Markdown()
                    with gr.Tab('Future Conditions to Watch', visible = True) as futConditPage4:
                        contentFutCondit4 = gr.Markdown()
                    with gr.Tab('Patient Information', visible = True) as patientInfoPage4:
                        contentPatientInfo4 = gr.Markdown()

                    
                    update_content4 = functools.partial(update_content, page_num=4)
                    
                    show_summaries_button = gr.Button("Show Summaries")
                    show_summaries_button.click(fn=update_content4, inputs=None, outputs=[contentDiag4, contentRx4, contentProc4, contentLab4, contentEMR4, contentPrevCondit4, contentFutCondit4, contentPatientInfo4])
                    
                    hide_summaries_button = gr.Button("Hide Summaries")
                    hide_summaries_button.click(fn=hide_content, inputs=None, outputs=[contentDiag4, contentRx4, contentProc4, contentLab4, contentEMR4, contentPrevCondit4, contentFutCondit4, contentPatientInfo4])
                           
            with gr.Column():
                patient4_chat = gr.Chatbot(label="patient4_chatbot", height=800)
                with gr.Row():
                    text_input = gr.Textbox(label="Your Message")
                    send_button = gr.Button("Send")
                respond4 = functools.partial(respond, thread_id="4")
                send_button.click(fn=respond4, inputs=[text_input, patient4_chat], outputs=[text_input, patient4_chat])

        
        with gr.Tab('Patient 5', visible = False, ) as tab5:
            with gr.Column():
                with gr.Tabs() as pageTabs5:
                    with gr.Tab('Diagnosis Summary', visible = True) as diagPage5:
                        contentDiag5 = gr.Markdown()
                    with gr.Tab('Rx Summary', visible = True) as rxPage5:
                        contentRx5 = gr.Markdown()
                    with gr.Tab('Procedure Summary', visible = True) as procPage5:
                        contentProc5 = gr.Markdown()
                    with gr.Tab('Lab Summary', visible = True) as labPage5:
                        contentLab5 = gr.Markdown()
                    with gr.Tab('EMR Summary', visible = True) as emrPage5:
                        contentEMR5 = gr.Markdown()
                    with gr.Tab('Previous Conditions', visible = True) as prevConditPage5:
                        contentPrevCondit5 = gr.Markdown()
                    with gr.Tab('Future Conditions to Watch', visible = True) as futConditPage5:
                        contentFutCondit5 = gr.Markdown()
                    with gr.Tab('Patient Information', visible = True) as patientInfoPage5:
                        contentPatientInfo5 = gr.Markdown()

                    update_content5 = functools.partial(update_content, page_num=5)
                    
                    show_summaries_button = gr.Button("Show Summaries")
                    show_summaries_button.click(fn=update_content5, inputs=None, outputs=[contentDiag5, contentRx5, contentProc5, contentLab5, contentEMR5, contentPrevCondit5, contentFutCondit5, contentPatientInfo5])
                    
                    hide_summaries_button = gr.Button("Hide Summaries")
                    hide_summaries_button.click(fn=hide_content, inputs=None, outputs=[contentDiag5, contentRx5, contentProc5, contentLab5, contentEMR5, contentPrevCondit5, contentFutCondit5, contentPatientInfo5])
                           
            with gr.Column():
                patient5_chat = gr.Chatbot(label="patient5_chatbot", height=800)
                with gr.Row():
                    text_input = gr.Textbox(label="Your Message")
                    send_button = gr.Button("Send")
                respond5 = functools.partial(respond, thread_id="5")
                send_button.click(fn=respond5, inputs=[text_input, patient5_chat], outputs=[text_input, patient5_chat])
            
                    
    user_input = gr.Textbox(label="Enter mcid for new chat:")
    new_chat_btn = gr.Button("New Chat")
    load_prev_chat_btn = gr.Button("Load Saved Chat")
    
    new_chat_btn.click(fn=show_next_tab, inputs=user_input, outputs=[tab1, tab2, tab3, tab4, tab5])
    load_prev_chat_btn.click(fn=show_next_tab_load,inputs=user_input, outputs=[tab1, tab2, tab3, tab4, tab5])
    
demo.launch(share=True)